# < QUBO Matrix Calculator >

This calculator prints out the d-wave quantum annealer command for solving a system of linear equations using the quadratic unconstrained binary optimization formulation. 

## Base 2 digit representation

* base_digit: the number of qubits used to represent a variable using base-2 representation
* variable_num: the number of variables used in the system of linear equations

In [153]:
base_digit = 16 #qubit digit using base-2 representation
variable_num = 2 #x,y,z

### Define qubit relations 

We implement the following two relations among the set of qubits utilized in the QUBO formulation:
   * Relation (a): q_i^2 = q_i (binary qubit)
   * Relation (b): q_i^+ * q_j^- = 0 (positive digits and negative digits multiply to 0)

In [154]:
local_var = var(','.join('q_%s_%s'%(i,j) for i in range(0,variable_num) for j in range(0,base_digit*2)))

S = PolynomialRing(QQ, len(local_var), local_var)
local_var = S.gens()

S1 = S.quotient([var^2-var for var in S.gens()])
local_var = S1.gens()

#Base 2 representation in S1
y_sub = []
for i in range(0,variable_num):
    var_local = i*base_digit*2
    positive_digit = sum([(2**j)*local_var[var_local+j] for j in range(0,base_digit)])
    negative_digit = sum([(2**j)*local_var[var_local+base_digit+j] for j in range(0,base_digit)])
    y_sub += [positive_digit - negative_digit]

#Ensure the products of positive/negative digit qubits are 0.
qubit_product = []
for i in range(0,variable_num):
    var_local = i*base_digit*2
    for j in range(0,base_digit):
        for k in range(base_digit,base_digit*2):
            qubit_product += [local_var[var_local+j]*local_var[var_local+k]]

S2 = S1.quotient(qubit_product)
local_var = S2.gens()

#Base 2 representation in S2
y = []
for i in range(0,variable_num):
    var_local = i*base_digit*2
    positive_digit = sum([(2**j)*local_var[var_local+j] for j in range(0,base_digit)])
    negative_digit = sum([(2**j)*local_var[var_local+base_digit+j] for j in range(0,base_digit)])
    y += [positive_digit - negative_digit]

## Systems of linear equations

### Define system of linear equations Ax = b

An exemplary system of linear equations for 2, 3, 4, 5, and 6 variables are shown.
* A: the matrix defining the system of linear equations Ax = b
* b: the image of the system of linear equations Ax = b
* D, R: the diagonal matrix and an invertible matrix which satisfies D = R^T A^T A R.

In [155]:
#System of linear equations Ax = b for 2 variables
A = matrix(QQ, [[3,1],[-1,2]])
b = matrix(QQ, [[-1],[5]])

D = matrix(QQ, [[8/5,0],[0,98/125]])
R = matrix(QQ, [[2/5,-1/25],[0,2/5]])
                
A_list = A.numpy()
b_list = b.numpy()
D_list = D.numpy()
R_list = R.numpy()

In [ ]:
#System of linear equations Ax = b for 3 variables

#A = matrix(QQ, [[5,0,1],
#                [-1,2,1],
#               [3,2,3]])
#b = matrix(QQ, [[-65], [-15], [-80]])

#Diagonalize matrix
#AtA = A.transpose()*A
#M = QuadraticForm(AtA)
#D,R = M.rational_diagonal_form(return_matrix=True)
#Process Matrices
#D = 2*D.Gram_matrix()

#A_list = A.numpy()
#b_list = b.numpy()
#D_list = D.numpy()
#R_list = R.numpy()

In [ ]:
#System of linear equations Ax = b for 4 variables

#A = matrix(QQ, [[5,0,1,3],
#                [-1,2,1,4],
#               [3,2,3,5],
#               [6,29,30,-14]])
#b = matrix(QQ, [[-65], [-15], [-80],[-1]])

#Diagonalize matrix
#AtA = A.transpose()*A
#M = QuadraticForm(AtA)
#D,R = M.rational_diagonal_form(return_matrix=True)
#Process Matrices
#D = 2*D.Gram_matrix()

#A_list = A.numpy()
#b_list = b.numpy()
#D_list = D.numpy()
#R_list = R.numpy()

In [ ]:
#System of linear equations Ax = b for 5 variables

#A = matrix(QQ, [[5,0,1,3,8],
#                [-1,2,1,4,5],
#               [3,2,3,5,26],
#               [6,29,30,-14,-1],
#               [5,3,4,-2,65]])
#b = matrix(QQ, [[-65], [-15], [-80],[-1],[47]])

#Diagonalize matrix
#AtA = A.transpose()*A
#M = QuadraticForm(AtA)
#D,R = M.rational_diagonal_form(return_matrix=True)
#Process Matrices
#D = 2*D.Gram_matrix()

#A_list = A.numpy()
#b_list = b.numpy()
#D_list = D.numpy()
#R_list = R.numpy()

In [ ]:
#System of linear equations Ax = b for 6 variables

#A = matrix(QQ, [[5,0,1,3,8,26],
#                [-1,2,1,4,5,-1],
#               [3,2,3,5,26,35],
#               [6,29,30,-14,-1,-4],
#               [5,3,4,-2,65,-3]])
#b = matrix(QQ, [[-65], [-15], [-80],[-1],[47]])

#Diagonalize matrix
#AtA = A.transpose()*A
#M = QuadraticForm(AtA)
#D,R = M.rational_diagonal_form(return_matrix=True)
#Process Matrices
#D = 2*D.Gram_matrix()

#A_list = A.numpy()
#b_list = b.numpy()
#D_list = D.numpy()
#R_list = R.numpy()

### Define base-2 representations

We define the vector of unknown variables using base-2 representations.
* mat_y_sub: base-2 representation obtained from only using relation (a)
* mat_y: base-2 representation obtained from using both relation (a) and (b)

In [156]:
mat_y_sub = (matrix(S1,y_sub)).transpose()
mat_y = (matrix(S2,y)).transpose()

### Convert matrices to impose qubit relations

In [157]:
#matrix A
mat_a_sub = matrix(S1,A_list)
mat_a = matrix(S2,A_list)

#vector b
mat_b_sub = matrix(S1,b_list)
mat_b = matrix(S2,b_list)

#matrix D
mat_d_sub = matrix(S1,D_list)
mat_d = matrix(S2,D_list)

#matrix R
mat_r_sub = matrix(S1,R_list)
mat_r = matrix(S2,R_list)

## QUBO matrix with Sylvester's law of inertia

This part of the code prints out the d-wave system command for solving a system of linear equations using the QUBO formulation which uses:
* Sylvester's law of inertia (i.e. matrices D and R are given)
* Relations (a) and (b).

In [158]:
F_new = ((mat_y.transpose()*mat_d*mat_y - 2*(mat_y.transpose()*mat_r.transpose()*mat_a.transpose()*mat_b))[0])[0]
G_new = str(expand(F_new))
G_new = G_new.replace("bar","")
print("QUBO Model")
print(G_new)

f_new = S(G_new)
new_qubo_monomial_power = f_new.exponents()
new_qubo_monomial_terms = f_new.dict()
new_qubo_matrix = Matrix(QQ, len(local_var))
for powers in new_qubo_monomial_power:
    coeff = new_qubo_monomial_terms.get(powers)
    index_list = []
    for k in range(0,len(powers)):
        if powers[k] == 1:
            index_list += [k]
    if len(index_list) == 1:
        matrix_index_1 = index_list[0]
        matrix_index_2 = index_list[0]
    else:
        matrix_index_1 = index_list[0]
        matrix_index_2 = index_list[1]
    new_qubo_matrix[matrix_index_1, matrix_index_2] = coeff

print("New QUBO Matrix")
print(new_qubo_matrix)

QUBO Model
32/5*q_0_0*q_0_1 + 64/5*q_0_0*q_0_2 + 128/5*q_0_1*q_0_2 + 128/5*q_0_0*q_0_3 + 256/5*q_0_1*q_0_3 + 512/5*q_0_2*q_0_3 + 256/5*q_0_0*q_0_4 + 512/5*q_0_1*q_0_4 + 1024/5*q_0_2*q_0_4 + 2048/5*q_0_3*q_0_4 + 512/5*q_0_0*q_0_5 + 1024/5*q_0_1*q_0_5 + 2048/5*q_0_2*q_0_5 + 4096/5*q_0_3*q_0_5 + 8192/5*q_0_4*q_0_5 + 1024/5*q_0_0*q_0_6 + 2048/5*q_0_1*q_0_6 + 4096/5*q_0_2*q_0_6 + 8192/5*q_0_3*q_0_6 + 16384/5*q_0_4*q_0_6 + 32768/5*q_0_5*q_0_6 + 2048/5*q_0_0*q_0_7 + 4096/5*q_0_1*q_0_7 + 8192/5*q_0_2*q_0_7 + 16384/5*q_0_3*q_0_7 + 32768/5*q_0_4*q_0_7 + 65536/5*q_0_5*q_0_7 + 131072/5*q_0_6*q_0_7 + 4096/5*q_0_0*q_0_8 + 8192/5*q_0_1*q_0_8 + 16384/5*q_0_2*q_0_8 + 32768/5*q_0_3*q_0_8 + 65536/5*q_0_4*q_0_8 + 131072/5*q_0_5*q_0_8 + 262144/5*q_0_6*q_0_8 + 524288/5*q_0_7*q_0_8 + 8192/5*q_0_0*q_0_9 + 16384/5*q_0_1*q_0_9 + 32768/5*q_0_2*q_0_9 + 65536/5*q_0_3*q_0_9 + 131072/5*q_0_4*q_0_9 + 262144/5*q_0_5*q_0_9 + 524288/5*q_0_6*q_0_9 + 1048576/5*q_0_7*q_0_9 + 2097152/5*q_0_8*q_0_9 + 16384/5*q_0_0*q_0_10 + 3

In [145]:
QM = new_qubo_matrix
# Print Python code for the run in D-Wave quantum processing unit
qubits = base_digit
Dimension = variable_num

print("\n\nfrom dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)") 



from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):8, ('q2','q2'):96/5, ('q3','q3'):256/5, ('q4','q4'):768/5, ('q5','q5'):512, ('q6','q6'):9216/5, ('q7','q7'):34816/5, ('q8','q8'):135168/5, ('q9','q9'):106496, ('q10','q10'):2113536/5, ('q11','q11'):8421376/5, ('q12','q12'):33619968/5, ('q13','q13'):26869760, ('q14','q14'):537133056/5, ('q15','q15'):2148007936/5, ('q16','q16'):8590983168/5, ('q17','q17'):6872367104, ('q18','q18'):137443147776/5, ('q19','q19'):549764202496/5, ('q20','q20'):2199040032768/5, ('q21','q21'):1759225315328, ('q22','q22'):35184439197696/5, ('q23','q23'):140737622573056/5, ('q24','q24'):562950221856768/5, ('q25','q25'):450360070111232, ('q26','q26'):9007200328482816/5, ('q27','q27'):36028799166447616/5, ('q28','q28'):144115192370823168/5, ('q29','q29'):115292152178671616, ('q30','q30'):2305843026393563136/5, ('q31','q31'):9223372071214514176/5, ('q32','q32'):368

quadratic = {('q1','q2'):32/5, ('q1','q3'):64/5, ('q1','q4'):128/5, ('q1','q5'):256/5, ('q1','q6'):512/5, ('q1','q7'):1024/5, ('q1','q8'):2048/5, ('q1','q9'):4096/5, ('q1','q10'):8192/5, ('q1','q11'):16384/5, ('q1','q12'):32768/5, ('q1','q13'):65536/5, ('q1','q14'):131072/5, ('q1','q15'):262144/5, ('q1','q16'):524288/5, ('q1','q17'):1048576/5, ('q1','q18'):2097152/5, ('q1','q19'):4194304/5, ('q1','q20'):8388608/5, ('q1','q21'):16777216/5, ('q1','q22'):33554432/5, ('q1','q23'):67108864/5, ('q1','q24'):134217728/5, ('q1','q25'):268435456/5, ('q1','q26'):536870912/5, ('q1','q27'):1073741824/5, ('q1','q28'):2147483648/5, ('q1','q29'):4294967296/5, ('q1','q30'):8589934592/5, ('q1','q31'):17179869184/5, ('q1','q32'):34359738368/5, ('q1','q33'):68719476736/5, ('q1','q34'):137438953472/5, ('q2','q3'):128/5, ('q2','q4'):256/5, ('q2','q5'):512/5, ('q2','q6'):1024/5, ('q2','q7'):2048/5, ('q2','q8'):4096/5, ('q2','q9'):8192/5, ('q2','q10'):16384/5, ('q2','q11'):32768/5, ('q2','q12'):65536/5, ('q2'

58','q59'):2251799813685248/5, ('q58','q60'):4503599627370496/5, ('q58','q61'):9007199254740992/5, ('q58','q62'):18014398509481984/5, ('q58','q63'):36028797018963968/5, ('q58','q64'):72057594037927936/5, ('q58','q65'):144115188075855872/5, ('q58','q66'):288230376151711744/5, ('q58','q67'):576460752303423488/5, ('q58','q68'):1152921504606846976/5, ('q59','q60'):9007199254740992/5, ('q59','q61'):18014398509481984/5, ('q59','q62'):36028797018963968/5, ('q59','q63'):72057594037927936/5, ('q59','q64'):144115188075855872/5, ('q59','q65'):288230376151711744/5, ('q59','q66'):576460752303423488/5, ('q59','q67'):1152921504606846976/5, ('q59','q68'):2305843009213693952/5, ('q60','q61'):36028797018963968/5, ('q60','q62'):72057594037927936/5, ('q60','q63'):144115188075855872/5, ('q60','q64'):288230376151711744/5, ('q60','q65'):576460752303423488/5, ('q60','q66'):1152921504606846976/5, ('q60','q67'):2305843009213693952/5, ('q60','q68'):4611686018427387904/5, ('q61','q62'):144115188075855872/5, ('q61

106','q119'):102760448/125, ('q106','q120'):205520896/125, ('q106','q121'):411041792/125, ('q106','q122'):822083584/125, ('q106','q123'):1644167168/125, ('q106','q124'):3288334336/125, ('q106','q125'):6576668672/125, ('q106','q126'):13153337344/125, ('q106','q127'):26306674688/125, ('q106','q128'):52613349376/125, ('q106','q129'):105226698752/125, ('q106','q130'):210453397504/125, ('q106','q131'):420906795008/125, ('q106','q132'):841813590016/125, ('q106','q133'):1683627180032/125, ('q106','q134'):3367254360064/125, ('q106','q135'):6734508720128/125, ('q106','q136'):13469017440256/125, ('q107','q108'):100352/125, ('q107','q109'):200704/125, ('q107','q110'):401408/125, ('q107','q111'):802816/125, ('q107','q112'):1605632/125, ('q107','q113'):3211264/125, ('q107','q114'):6422528/125, ('q107','q115'):12845056/125, ('q107','q116'):25690112/125, ('q107','q117'):51380224/125, ('q107','q118'):102760448/125, ('q107','q119'):205520896/125, ('q107','q120'):411041792/125, ('q107','q121'):822083584

In [27]:
#print out latex command
new_qubo_matrix_11 = new_qubo_matrix.matrix_from_rows_and_columns(range(0,base_digit), range(0,base_digit))
print(latex(new_qubo_matrix_11))

\left(\begin{array}{rrrrrrrrrr}
1302 & 168 & 336 & 672 & 1344 & 2688 & 5376 & 10752 & 21504 & 43008 \\
0 & 2688 & 672 & 1344 & 2688 & 5376 & 10752 & 21504 & 43008 & 86016 \\
0 & 0 & 5712 & 2688 & 5376 & 10752 & 21504 & 43008 & 86016 & 172032 \\
0 & 0 & 0 & 12768 & 10752 & 21504 & 43008 & 86016 & 172032 & 344064 \\
0 & 0 & 0 & 0 & 30912 & 43008 & 86016 & 172032 & 344064 & 688128 \\
0 & 0 & 0 & 0 & 0 & 83328 & 172032 & 344064 & 688128 & 1376256 \\
0 & 0 & 0 & 0 & 0 & 0 & 252672 & 688128 & 1376256 & 2752512 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 849408 & 2752512 & 5505024 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 3075072 & 11010048 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 11655168
\end{array}\right)


## QUBO matrix without Sylvester's law of inertia but applying qubit product formula

This part of the code prints out the d-wave system command for solving a system of linear equations using the QUBO formulation which uses:
* Relations (a) and (b).

In [28]:
F = ((mat_y.transpose()*mat_a.transpose()*mat_a*mat_y - 2*(mat_y.transpose()*mat_a.transpose()*mat_b))[0])[0]
print("QUBO Model")
G = str(expand(F))
G = G.replace("bar","")
print(G)

f = S(G)
qubo_monomial_power = f.exponents()
qubo_monomial_terms = f.dict()
qubo_matrix = Matrix(QQ, len(local_var))
for powers in qubo_monomial_power:
    coeff = qubo_monomial_terms.get(powers)
    index_list = []
    for k in range(0,len(powers)):
        if powers[k] == 1:
            index_list += [k]
    if len(index_list) == 1:
        matrix_index_1 = index_list[0]
        matrix_index_2 = index_list[0]
    else:
        matrix_index_1 = index_list[0]
        matrix_index_2 = index_list[1]
    qubo_matrix[matrix_index_1, matrix_index_2] = coeff

print("Mid QUBO Matrix")
print(qubo_matrix)

QUBO Model
168*q_0_0*q_0_1 + 336*q_0_0*q_0_2 + 672*q_0_1*q_0_2 + 672*q_0_0*q_0_3 + 1344*q_0_1*q_0_3 + 2688*q_0_2*q_0_3 + 1344*q_0_0*q_0_4 + 2688*q_0_1*q_0_4 + 5376*q_0_2*q_0_4 + 10752*q_0_3*q_0_4 + 2688*q_0_0*q_0_5 + 5376*q_0_1*q_0_5 + 10752*q_0_2*q_0_5 + 21504*q_0_3*q_0_5 + 43008*q_0_4*q_0_5 + 5376*q_0_0*q_0_6 + 10752*q_0_1*q_0_6 + 21504*q_0_2*q_0_6 + 43008*q_0_3*q_0_6 + 86016*q_0_4*q_0_6 + 172032*q_0_5*q_0_6 + 10752*q_0_0*q_0_7 + 21504*q_0_1*q_0_7 + 43008*q_0_2*q_0_7 + 86016*q_0_3*q_0_7 + 172032*q_0_4*q_0_7 + 344064*q_0_5*q_0_7 + 688128*q_0_6*q_0_7 + 21504*q_0_0*q_0_8 + 43008*q_0_1*q_0_8 + 86016*q_0_2*q_0_8 + 172032*q_0_3*q_0_8 + 344064*q_0_4*q_0_8 + 688128*q_0_5*q_0_8 + 1376256*q_0_6*q_0_8 + 2752512*q_0_7*q_0_8 + 43008*q_0_0*q_0_9 + 86016*q_0_1*q_0_9 + 172032*q_0_2*q_0_9 + 344064*q_0_3*q_0_9 + 688128*q_0_4*q_0_9 + 1376256*q_0_5*q_0_9 + 2752512*q_0_6*q_0_9 + 5505024*q_0_7*q_0_9 + 11010048*q_0_8*q_0_9 + 168*q_0_10*q_0_11 + 336*q_0_10*q_0_12 + 672*q_0_11*q_0_12 + 672*q_0_10*q_0_13 + 13

In [29]:
#print out latex command
qubo_matrix_11 = new_qubo_matrix.matrix_from_rows_and_columns(range(0,base_digit), range(0,base_digit))
print(latex(qubo_matrix_11))

\left(\begin{array}{rrrrrrrrrr}
1302 & 168 & 336 & 672 & 1344 & 2688 & 5376 & 10752 & 21504 & 43008 \\
0 & 2688 & 672 & 1344 & 2688 & 5376 & 10752 & 21504 & 43008 & 86016 \\
0 & 0 & 5712 & 2688 & 5376 & 10752 & 21504 & 43008 & 86016 & 172032 \\
0 & 0 & 0 & 12768 & 10752 & 21504 & 43008 & 86016 & 172032 & 344064 \\
0 & 0 & 0 & 0 & 30912 & 43008 & 86016 & 172032 & 344064 & 688128 \\
0 & 0 & 0 & 0 & 0 & 83328 & 172032 & 344064 & 688128 & 1376256 \\
0 & 0 & 0 & 0 & 0 & 0 & 252672 & 688128 & 1376256 & 2752512 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 849408 & 2752512 & 5505024 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 3075072 & 11010048 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 11655168
\end{array}\right)


## QUBO matrix without Sylvester's law of inertia and no qubit product formula

This part of the code prints out the d-wave system command for solving a system of linear equations using the QUBO formulation which only uses:
* Relation (a).

In [159]:
F_vanilla = ((mat_y_sub.transpose()*mat_a_sub.transpose()*mat_a_sub*mat_y_sub - 2*(mat_y_sub.transpose()*mat_a_sub.transpose()*mat_b_sub))[0])[0]
print("QUBO Model")
G_vanilla = str(expand(F_vanilla))
G_vanilla = G_vanilla.replace("bar","")
print(G_vanilla)

f_vanilla = S(G_vanilla)
vanilla_qubo_monomial_power = f_vanilla.exponents()
vanilla_qubo_monomial_terms = f_vanilla.dict()
vanilla_qubo_matrix = Matrix(QQ, len(local_var))
for powers in vanilla_qubo_monomial_power:
    coeff = vanilla_qubo_monomial_terms.get(powers)
    index_list = []
    for k in range(0,len(powers)):
        if powers[k] == 1:
            index_list += [k]
    if len(index_list) == 1:
        matrix_index_1 = index_list[0]
        matrix_index_2 = index_list[0]
    else:
        matrix_index_1 = index_list[0]
        matrix_index_2 = index_list[1]
    vanilla_qubo_matrix[matrix_index_1, matrix_index_2] = coeff

print("Vanilla QUBO Matrix")
print(vanilla_qubo_matrix)

QUBO Model
40*q_0_0*q_0_1 + 80*q_0_0*q_0_2 + 160*q_0_1*q_0_2 + 160*q_0_0*q_0_3 + 320*q_0_1*q_0_3 + 640*q_0_2*q_0_3 + 320*q_0_0*q_0_4 + 640*q_0_1*q_0_4 + 1280*q_0_2*q_0_4 + 2560*q_0_3*q_0_4 + 640*q_0_0*q_0_5 + 1280*q_0_1*q_0_5 + 2560*q_0_2*q_0_5 + 5120*q_0_3*q_0_5 + 10240*q_0_4*q_0_5 + 1280*q_0_0*q_0_6 + 2560*q_0_1*q_0_6 + 5120*q_0_2*q_0_6 + 10240*q_0_3*q_0_6 + 20480*q_0_4*q_0_6 + 40960*q_0_5*q_0_6 + 2560*q_0_0*q_0_7 + 5120*q_0_1*q_0_7 + 10240*q_0_2*q_0_7 + 20480*q_0_3*q_0_7 + 40960*q_0_4*q_0_7 + 81920*q_0_5*q_0_7 + 163840*q_0_6*q_0_7 + 5120*q_0_0*q_0_8 + 10240*q_0_1*q_0_8 + 20480*q_0_2*q_0_8 + 40960*q_0_3*q_0_8 + 81920*q_0_4*q_0_8 + 163840*q_0_5*q_0_8 + 327680*q_0_6*q_0_8 + 655360*q_0_7*q_0_8 + 10240*q_0_0*q_0_9 + 20480*q_0_1*q_0_9 + 40960*q_0_2*q_0_9 + 81920*q_0_3*q_0_9 + 163840*q_0_4*q_0_9 + 327680*q_0_5*q_0_9 + 655360*q_0_6*q_0_9 + 1310720*q_0_7*q_0_9 + 2621440*q_0_8*q_0_9 + 20480*q_0_0*q_0_10 + 40960*q_0_1*q_0_10 + 81920*q_0_2*q_0_10 + 163840*q_0_3*q_0_10 + 327680*q_0_4*q_0_10 + 65

In [160]:
QM = vanilla_qubo_matrix
# Print Python code for the run in D-Wave quantum processing unit
qubits = base_digit
Dimension = variable_num

print("\n\nfrom dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)") 



from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26, ('q2','q2'):72, ('q3','q3'):224, ('q4','q4'):768, ('q5','q5'):2816, ('q6','q6'):10752, ('q7','q7'):41984, ('q8','q8'):165888, ('q9','q9'):659456, ('q10','q10'):2629632, ('q11','q11'):10502144, ('q12','q12'):41975808, ('q13','q13'):167837696, ('q14','q14'):671219712, ('q15','q15'):2684616704, ('q16','q16'):10737942528, ('q17','q17'):-6, ('q18','q18'):8, ('q19','q19'):96, ('q20','q20'):512, ('q21','q21'):2304, ('q22','q22'):9728, ('q23','q23'):39936, ('q24','q24'):161792, ('q25','q25'):651264, ('q26','q26'):2613248, ('q27','q27'):10469376, ('q28','q28'):41910272, ('q29','q29'):167706624, ('q30','q30'):670957568, ('q31','q31'):2684092416, ('q32','q32'):10736893952, ('q33','q33'):-13, ('q34','q34'):-16, ('q35','q35'):8, ('q36','q36'):176, ('q37','q37'):992, ('q38','q38'):4544, ('q39','q39'):19328, ('q40','q40'):79616, ('q41','q41'):323

quadratic = {('q1','q2'):40, ('q1','q3'):80, ('q1','q4'):160, ('q1','q5'):320, ('q1','q6'):640, ('q1','q7'):1280, ('q1','q8'):2560, ('q1','q9'):5120, ('q1','q10'):10240, ('q1','q11'):20480, ('q1','q12'):40960, ('q1','q13'):81920, ('q1','q14'):163840, ('q1','q15'):327680, ('q1','q16'):655360, ('q1','q17'):-20, ('q1','q18'):-40, ('q1','q19'):-80, ('q1','q20'):-160, ('q1','q21'):-320, ('q1','q22'):-640, ('q1','q23'):-1280, ('q1','q24'):-2560, ('q1','q25'):-5120, ('q1','q26'):-10240, ('q1','q27'):-20480, ('q1','q28'):-40960, ('q1','q29'):-81920, ('q1','q30'):-163840, ('q1','q31'):-327680, ('q1','q32'):-655360, ('q1','q33'):2, ('q1','q34'):4, ('q1','q35'):8, ('q1','q36'):16, ('q1','q37'):32, ('q1','q38'):64, ('q1','q39'):128, ('q1','q40'):256, ('q1','q41'):512, ('q1','q42'):1024, ('q1','q43'):2048, ('q1','q44'):4096, ('q1','q45'):8192, ('q1','q46'):16384, ('q1','q47'):32768, ('q1','q48'):65536, ('q1','q49'):-2, ('q1','q50'):-4, ('q1','q51'):-8, ('q1','q52'):-16, ('q1','q53'):-32, ('q1','q54

In [31]:
#print out latex command
qubo_matrix_11 = vanilla_qubo_matrix.matrix_from_rows_and_columns(range(0,base_digit), range(0,base_digit))
print(latex(qubo_matrix_11))

\left(\begin{array}{rrrrrrrrrr}
1302 & 168 & 336 & 672 & 1344 & 2688 & 5376 & 10752 & 21504 & 43008 \\
0 & 2688 & 672 & 1344 & 2688 & 5376 & 10752 & 21504 & 43008 & 86016 \\
0 & 0 & 5712 & 2688 & 5376 & 10752 & 21504 & 43008 & 86016 & 172032 \\
0 & 0 & 0 & 12768 & 10752 & 21504 & 43008 & 86016 & 172032 & 344064 \\
0 & 0 & 0 & 0 & 30912 & 43008 & 86016 & 172032 & 344064 & 688128 \\
0 & 0 & 0 & 0 & 0 & 83328 & 172032 & 344064 & 688128 & 1376256 \\
0 & 0 & 0 & 0 & 0 & 0 & 252672 & 688128 & 1376256 & 2752512 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 849408 & 2752512 & 5505024 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 3075072 & 11010048 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 11655168
\end{array}\right)


## Export matrix to .txt file

In [22]:
qubo_matrix_output = str(new_qubo_matrix)
text_file = open("new_qubo_matrix.txt", "w")
text_file.write(qubo_matrix_output)
text_file.close()

qubo_matrix_output = str(qubo_matrix)
text_file = open("mid_qubo_matrix.txt", "w")
text_file.write(qubo_matrix_output)
text_file.close()

qubo_matrix_output = str(vanilla_qubo_matrix)
text_file = open("vanilla_qubo_matrix.txt", "w")
text_file.write(qubo_matrix_output)
text_file.close()

# <Sylvester's theorem Example>

This example shows how sage can compute matrices D and R which satisfies

D = R^T A^T A R

for any matrix A.

## 3x3 matrix with Rank 2 example: infinitely many solutions

In [1]:
A = matrix(QQ, [[5,0,1],
                [-1,2,1],
               [4,2,2]])
b = matrix(QQ, [[-65], [-15], [-80]])
print("Goal: Solve the linear system of equations Ax = b")
print("Matrix A:")
print(A)
print("Matrix b:")
print(b)

Goal: Solve the linear system of equations Ax = b
Matrix A:
[ 5  0  1]
[-1  2  1]
[ 4  2  2]
Matrix b:
[-65]
[-15]
[-80]


In [2]:
print("Apply Sylvester's Law of Inertia to A^TA")
AtA = A.transpose()*A
M = QuadraticForm(AtA)
D,R = M.rational_diagonal_form(return_matrix=True)

#Process Matrices
D = 2*D.Gram_matrix()

print("Diagonal Matrix D:")
print(D)
print("Non-singular change of basis R:")
print(R)

Apply Sylvester's Law of Inertia to A^TA
Diagonal Matrix D:
[  42    0    0]
[   0 50/7    0]
[   0    0    0]
Non-singular change of basis R:
[  55/139 -265/973    7/139]
[-252/139   85/139   21/139]
[ 420/139   90/139  -35/139]


In [3]:
b.transpose()*b

[10850]

In [4]:
print("Check whether the solution Ax = b is obtainable:")
print("Check inner product <b, A*R*e_3>")
e_1 = matrix(QQ, [[1],[0],[0]])
e_2 = matrix(QQ, [[0],[1],[0]])
b_1 = A*R*e_1 
b_2 = A*R*e_2
a_1 = (b.transpose()*b_1)[0]/(b_1.transpose()*b_1)[0]
a_2 = (b.transpose()*b_2)[0]/(b_2.transpose()*b_2)[0]
print(a_1[0])
print(a_2[0])
crit = (b == a_1*b_1 + a_2*b_2)
if crit:
    print("Pass")
else:
    print("Fail")

Check whether the solution Ax = b is obtainable:
Check inner product <b, A*R*e_3>
-15
-14
Pass


In [5]:
x = matrix(QQ, [[1],[28],[-70]])
print("Solution x:")
print(x)
print("Ax = b:")
print(A*x - b)

Solution x:
[  1]
[ 28]
[-70]
Ax = b:
[0]
[0]
[0]


In [6]:
print("Change of Coordinates y = Rx:")
y = (R.inverse()*x)
print(y)

Change of Coordinates y = Rx:
[-15]
[-14]
[ 62]


In [7]:
print("Verify that setting x = R^-1 y solves the linear equation:")
x = R*y
print("Ax = b:")
print(A*x - b)

Verify that setting x = R^-1 y solves the linear equation:
Ax = b:
[0]
[0]
[0]


In [8]:
print("Create all basis which satisfy Ax = b using y:")
print("y = (-15, -14, k) for any real number k")
print("Test implement")
test_truth=True
for k in range(0,100):
    y = matrix(QQ,[[-15], [-14],[0]])
    x = R*y
    result = A*x - b
    result_norm = result.transpose()*result
    if result != 0:
        test_truth=False
        break
if test_truth:
    print("Test Passed")
else:
    print("Test Failed")
print("--------------------------")
print("Canonical Vector of form x_0 + k*v_1")
print("for some fixed base-point vector x_0,")
print("basis vector v_1,") 
print("and any real number k.")
y = matrix(QQ,[[-15], [-14],[0]])
v_1 = matrix(QQ, [[0],[0],[1]])
x = R*y
v_1 = R*v_1
print("x_0")
print(x)
print("v_1")
print(v_1)

Create all basis which satisfy Ax = b using y:
y = (-15, -14, k) for any real number k
Test implement
Test Passed
--------------------------
Canonical Vector of form x_0 + k*v_1
for some fixed base-point vector x_0,
basis vector v_1,
and any real number k.
x_0
[ -295/139]
[ 2590/139]
[-7560/139]
v_1
[  7/139]
[ 21/139]
[-35/139]


In [9]:
print("Reconstruct the sample vector")
print("Note that ")
print(x + 62*v_1)

Reconstruct the sample vector
Note that 
[  1]
[ 28]
[-70]


## 3x3 matrix with Rank 2 example: no solutions

In [10]:
A = matrix(QQ, [[5,0,1],
                [-1,2,1],
               [4,2,2]])
b_no = matrix(QQ, [[0], [1], [0]])
print("Goal: Solve the linear system of equations Ax = b")
print("Matrix A:")
print(A)
print("Matrix b:")
print(b_no)

Goal: Solve the linear system of equations Ax = b
Matrix A:
[ 5  0  1]
[-1  2  1]
[ 4  2  2]
Matrix b:
[0]
[1]
[0]


In [11]:
print("Apply Sylvester's Law of Inertia")
AtA = A.transpose()*A
M = QuadraticForm(AtA)
D,R = M.rational_diagonal_form(return_matrix=True)

#Process Matrices
D = 2*D.Gram_matrix()

print("Diagonal Matrix D:")
print(D)
print("Non-singular change of basis R:")
print(R)

Apply Sylvester's Law of Inertia
Diagonal Matrix D:
[  42    0    0]
[   0 50/7    0]
[   0    0    0]
Non-singular change of basis R:
[  55/139 -265/973    7/139]
[-252/139   85/139   21/139]
[ 420/139   90/139  -35/139]


In [12]:
print("Check whether the solution Ax = b is obtainable:")
print("Check inner product <b, A*R*e_3>")
e_1 = matrix(QQ, [[1],[0],[0]])
e_2 = matrix(QQ, [[0],[1],[0]])
b_1 = A*R*e_1 
b_2 = A*R*e_2
a_1 = (b_no.transpose()*b_1)[0]/(b_1.transpose()*b_1)[0]
a_2 = (b_no.transpose()*b_2)[0]/(b_2.transpose()*b_2)[0]
print(a_1[0])
print(a_2[0])
crit = (b_no == a_1*b_1 + a_2*b_2)
if crit:
    print("Pass")
else:
    print("Fail")

Check whether the solution Ax = b is obtainable:
Check inner product <b, A*R*e_3>
-1/42
3/10
Fail
